# <center>PRS computing</center>

This page documents the PRS computing for PGS catalog. It contain 3 steps: <br>1. PGScatalog download <br>2. Computing <br>3. File production<br><font color = red>You need to set up the working directory and provide the path for some data required for PRS computing</font>

## working directory and required file setup

1. Working Directory <br>&nbsp;&nbsp;&nbsp;&nbsp;PGS score files, PRS computing results and intermediate files will be generated under working directory <br> &nbsp;&nbsp;&nbsp;&nbsp; pgsJsonFile(prs/pgs_scores_data.json) is downloaded from https://www.pgscatalog.org/browse/scores/ <br> 2. PRS Compute <br>&nbsp;&nbsp;&nbsp;&nbsp;SJLIFE_CCSS_varfile is the compressed and indexed SJLIFE_CCSS variants <br>&nbsp;&nbsp;&nbsp;&nbsp;sample list file based on ethinicity under working directory ( sample.asa; sample.ceu; sample.yri) <br>&nbsp;&nbsp;&nbsp;&nbsp;pre-computed hwe test results <br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;/research/rgs01/resgen/legacy/gb_customTracks/tp/jwang/TASK/survivorship/HWE/combined/forprs/phwe.asa.gz<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;/research/rgs01/resgen/legacy/gb_customTracks/tp/jwang/TASK/survivorship/HWE/combined/forprs/phwe.ceu.gz<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;/research/rgs01/resgen/legacy/gb_customTracks/tp/jwang/TASK/survivorship/HWE/combined/forprs/phwe.yri.gz<br>3. File production <br>&nbsp;&nbsp;&nbsp;&nbsp;updateTime: The date when PRS was computed<br>&nbsp;&nbsp;&nbsp;&nbsp;pgs_all_metadata.xlsx Download from https://www.pgscatalog.org/downloads/<br>&nbsp;&nbsp;&nbsp;&nbsp;pgs_traits_data.json Download from https://www.pgscatalog.org/browse/traits/<br>&nbsp;&nbsp;&nbsp;&nbsp;pathToPRS: relative path of working directory to tp(jwang/TASK/prs/prsscore)

In [1]:
import re,os

workingDir = '/research/rgs01/resgen/legacy/gb_customTracks/tp/jwang/TASK/prs'

#####download
# pgs_scores_data.json downloaded from https://www.pgscatalog.org/browse/scores/
pgsJsonFile = os.path.join(workingDir,'pgs_scores_data.json')


### PRS Compute
#sampleID2IntegerID
def get_id2name(id2nameFile):
    id2name = {}
    fh = open(id2nameFile)
    for line in fh:
        l = line.strip().split('\t')
        id2name[l[1]] = l[0]
    fh.close()
    return id2name
#SJLIFE_CCSS_varfile
SJLIFE_CCSS_varfile = '/research/rgs01/resgen/legacy/gb_customTracks/tp/jwang/TASK/prs/SJLIFE.CCSS.variants.gz'
sampleIDmap = '/research/rgs01/resgen/legacy/gb_customTracks/tp/files/hg38/sjlife/clinical/samples.idmap'
#sample list file based on ethinicity
#ASA: sample.asa; CEU: sample.ceu; YRI: sample.yri (under working directory)
sampleFile = {p:os.path.join(workingDir,'sample.'+p) for p in ['asa','ceu','yri']}
#sample list file with integer ID
id2name = get_id2name(sampleIDmap)
for p in sampleFile:
    samples = [x.strip() for x in open(sampleFile[p])]
    idsamples = [id2name[x] for x in samples]
    sample_file = sampleFile[p]+'.intid'
    out = open(sample_file,'w')
    for intsam in idsamples:
        out.write(intsam+'\n')
    out.close()

#pre-computed hwe
hwedir = '/research/rgs01/resgen/legacy/gb_customTracks/tp/jwang/TASK/survivorship/HWE/combined/forprs'
hweTest = {p:os.path.join(hwedir,'phwe.'+p+'.gz') for p in ['asa','ceu','yri']}

### File production
#The following files are required (under working directory)
updateTime = '11/2022'
#pgs_all_metadata.xlsx Download from https://www.pgscatalog.org/downloads/
#pgs_traits_data.json Download from https://www.pgscatalog.org/browse/traits/
pgsMetadata = os.path.join(workingDir,'pgs_all_metadata.xlsx')
pgsTraits = os.path.join(workingDir,'pgs_traits_data.json')
#relative path to prs scores under tp ('jwang/TASK/prs/prsscore')
pathToPRS = 'jwang/TASK/prs'
tracktable = os.path.join(workingDir,'track_table.prs')
print('working directory and required file setup Done!')

working directory and required file setup Done!


## 1. PGScatalog download

In [3]:
import sys

if not os.path.isfile('pgsdownload.py'):
    print('Please run the "Tools used for prs computing" below to generate pgsdownload.py')
    sys.exit(1)

!bsub -q standard python3 pgsdownload.py $pgsJsonFile $workingDir


HPCF: WARNING! No Memory was requested!
      A default memory request of 2.50 GB has been placed for this job
      The job will be killed if   2.50 GB of memory is used
Job <173141494> is submitted to queue <standard>.


## 2. PRS Compute

In [10]:
import sys,os

if not os.path.isfile('prs_compute.py'):
    print('Please run the "Tools used for prs computing" below to generate prs_compute.py')
    sys.exit(1)

allPGSID = [x for x in os.listdir(workingDir) if os.path.isdir(os.path.join(workingDir,x)) and x.startswith('PGS')]
population = ['asa','ceu','yri']
for p in population:
    for pgs in allPGSID:
        scorefile = os.path.join(*[workingDir,pgs,pgs+'_hmPOS_GRCh38.txt.gz'])
        if not os.path.isfile(scorefile):
            print('Score file was not downloaded successfully: '+pgs)
            continue
        outprs = os.path.join(*[workingDir,pgs,pgs+'.'+p+'.prs'])
        outstat = os.path.join(*[workingDir,pgs,pgs+'.'+p+'.stat'])
        outmafprs = os.path.join(*[workingDir,pgs,pgs+'.'+p+'.maf'+'.prs'])
        outmafstat = os.path.join(*[workingDir,pgs,pgs+'.'+p+'.maf'+'.stat'])
        commandLine = 'bsub -q standard -M 3000 '
        commandLine += '-eo '+ os.path.join(*[workingDir,pgs,pgs+'.'+p+'.elog'])
        commandLine += ' -oo '+ os.path.join(*[workingDir,pgs,pgs+'.'+p+'.log'])
        commandLine += ' python3 prs_compute.py'
        commandLine +=  ' -s '+ scorefile
        commandLine +=  ' -d '+ os.path.join(workingDir,pgs)
        commandLine +=  ' --sample '+ sampleFile[p] 
        commandLine +=  ' --hwe '+ hweTest[p]
        commandLine +=  ' --ancestry '+ p
        commandLine += ' -v ' + SJLIFE_CCSS_varfile
        commandLine_maf = commandLine
        commandLine +=  ' -o ' + outprs
        commandLine +=  ' --statout '+ outstat
        if p == 'asa':
            commandLine +=  ' --matchout'
        os.system(commandLine)
        #maf filtered
        commandLine_maf += ' -o ' + outmafprs
        commandLine_maf += ' --statout '+ outmafstat
        commandLine_maf +=  ' --maf'
        os.system(commandLine_maf)
print('PRS computing Job submmited!')



Job <173256429> is submitted to queue <standard>.
Job <173256430> is submitted to queue <standard>.
Job <173256431> is submitted to queue <standard>.
Job <173256432> is submitted to queue <standard>.
Job <173256433> is submitted to queue <standard>.
Job <173256434> is submitted to queue <standard>.
Job <173256435> is submitted to queue <standard>.
Job <173256436> is submitted to queue <standard>.
Job <173256437> is submitted to queue <standard>.
Job <173256438> is submitted to queue <standard>.
Job <173256439> is submitted to queue <standard>.
Job <173256440> is submitted to queue <standard>.
PRS computing Job submmited!


## 3. File production

In [2]:
import sys,os
import re
import subprocess as sp

if not os.path.isfile('gentracktable.py'):
    print('Please run "Tools used for prs computing" below to generate gentracktable.py')
    sys.exit(1)
if not os.path.isfile('make_prs_db.bundle.js'):
    print('make_prs_db.bundle.js is not available!')
    sys.exit(1)
    
#Generate track table
log_trackTable = os.path.join(workingDir,'tracktable.log')
elog_trackTable = os.path.join(workingDir,'tracktable.elog')
command_trackTable = 'bsub -q standard'
command_trackTable += ' -eo ' + elog_trackTable
command_trackTable += ' -oo ' + log_trackTable
command_trackTable += ' python3 gentracktable.py'
command_trackTable += ' -m ' + pgsMetadata
command_trackTable += ' -t ' + pgsTraits
command_trackTable += ' -w ' + workingDir
command_trackTable += ' -o ' + tracktable
command_trackTable += ' -p ' + pathToPRS
jobRt = sp.run(command_trackTable, shell=True, stdout=sp.PIPE).stdout.decode('utf-8')
jobid = re.search('<(\d+?)>',jobRt).group(1)

#make prs db (tp/files/hg38/sjlife/clinical/PRS/)
log_prsdb = os.path.join(workingDir,'prsdb.log')
elog_prsdb = os.path.join(workingDir,'prsdb.elog')
command_prsdb = 'bsub -q standard -M 20000 -w "done('+jobid+')"'
command_prsdb += ' -eo ' + elog_prsdb
command_prsdb += ' -oo ' + log_prsdb
command_prsdb += ' node --max-old-space-size=20480 make_prs_db.bundle.js ' + tracktable + ' ' + updateTime
os.system(command_prsdb)



Job <173252812> is submitted to queue <standard>.


HPCF: WARNING! No Memory was requested!
      A default memory request of 2.50 GB has been placed for this job
      The job will be killed if   2.50 GB of memory is used


0

## Tools used for prs computing

In [1]:
%%writefile pgsdownload.py

#!/usr/bin/python3

import os
import sys
import json
import re

def EXTPGS(jsfile):
    with open(jsfile) as input:
        pgsJS = json.load(input)
    PGS = []
    for p in pgsJS['data']:
        PGS.append(re.search('PGS\d+',p['Polygenic Score ID & Name']).group())
    return PGS

def GETURL(pgs):
    return 'https://ftp.ebi.ac.uk/pub/databases/spot/pgs/scores/'+pgs+'/ScoringFiles/Harmonized/'+pgs+'_hmPOS_GRCh38.txt.gz'


PGSID = EXTPGS(sys.argv[1])
wd = sys.argv[2]

for pid in PGSID:
    url = GETURL(pid)
    pidDir = os.path.join(wd,pid)
    if not os.path.isdir(pidDir):
        os.system('mkdir '+pidDir)
    command_download = 'LSB_JOB_REPORT_MAIL=N bsub -q standard'
    command_download += ' -oo ' + os.path.join(pidDir,pid+'.download.log')
    command_download += ' -eo ' + os.path.join(pidDir,pid+'.download.elog')
    command_download += ' wget '+url+' -O '+os.path.join(pidDir,pid+'_hmPOS_GRCh38.txt.gz')
    os.system(command_download)


Overwriting pgsdownload.py


In [2]:
%%writefile prs_compute.py
#!/usr/bin/python3

####################################################
#
# Computing PRS Score                              #
#
####################################################

script_description="""
computing PRS score
inputfile: 
           pgs score file from PGS catalog (harmonized hg38)
           working directory
outputfile:
           
                        
"""



##########################################
# Initial QC of variant data
# functions for Initial QC of variant data 
##########################################

#check strand-ambiguous SNPs
def CK_AMB_SNP(a,b):
    a = a.upper()
    b = b.upper()
    AMBSNPS = {'G':'C','C':'G','A':'T','T':'A'}
    if a in AMBSNPS and AMBSNPS[a] == b:
        return True
    else:
        return False

def QCVAR(in_f,out_file):
	global strandAmbiguousVar,dupVar,sexChromosomeVar
	required_col = ['chr_name','chr_position','effect_allele','reference_allele','effect_weight']
	fileHeader = []
	fh = gzip.open(in_f)
	out = open(out_file+'.unsort','w')
	out.write('\t'.join(required_col)+'\n')
	SEARCHH = ['hm_chr','hm_pos','effect_allele'] #extract columns in this list
	SEARCHD = {}
	uniq_var = set()
	for line in fh:
		line = line.decode('utf-8')
		if line.startswith("#"):
			continue
		elif line.startswith('chr_name') or line.startswith('rsID'):
			fileHeader = line.strip().split('\t')
			if not 'other_allele' in fileHeader:
				SEARCHH.extend(['hm_inferOtherAllele','effect_weight'])
			else:
				SEARCHH.extend(['other_allele','effect_weight'])
			for e in SEARCHH:
				SEARCHD[e] = fileHeader.index(e)
			continue
		l = line.replace('\n','').split('\t')
		if not l[SEARCHD['hm_pos']] or not l[SEARCHD['hm_chr']]:
			continue
		if not re.search('^[0-9]',l[SEARCHD['hm_chr']]):
			sexChromosomeVar += 1 #remove non-autochromosome variant
			continue
		OUTL = [l[SEARCHD[e]] for e in SEARCHH]
		other_alleles = OUTL[3].split('/')
		effalle = OUTL[2]
		for e in other_alleles:
			var = '.'.join(OUTL[0:2]+[effalle,e])
			if len(effalle) != 1 or len(e) != 1: #remove indels
				continue
			if CK_AMB_SNP(effalle,e): #check strand-ambiguous SNPs
				strandAmbiguousVar += 1
				continue
			if var in uniq_var:
				dupVar += 1
				continue
			else:
				uniq_var.add(var)
			out.write('\t'.join(OUTL[0:3]+[e,OUTL[4]])+'\n')
	fh.close()
	out.close()
	os.system('head -n 1 '+out_file+'.unsort >'+out_file+';tail -n +2 '+out_file+'.unsort|sort -k1,1 -k2,2n >>'+out_file+';rm -f '+out_file+'.unsort')


#########################################
# split SNPs into chunks of 10,000 SNPs
########################################

def SNPSPLIT(f):
    fileDir,snpfile = os.path.split(f)
    os.system('tail -n +2 '+f+'| split -l 10000 - '+f+'_x')
    snpchunks = [x for x in os.listdir(fileDir) if x.startswith(snpfile+'_x') and not x.endswith('.region')]
    for snpchunk in snpchunks:
        snpchunkfile = os.path.join(fileDir,snpchunk)
        os.system('head -n 1 '+f+' >'+snpchunkfile+'.tmp')
        os.system('cat '+snpchunkfile+' >>'+snpchunkfile+'.tmp')
        os.system('mv '+snpchunkfile+'.tmp '+snpchunkfile)
        os.system('tail -n +2 '+snpchunkfile+'|awk \'BEGIN{FS=OFS="\t"} {print $1,$2,$2}\'  >'+snpchunkfile+'.region')
    return snpchunks


### functions for Filter for matching PGS and SJLIFE/CCSS variants
#get region from PRS_file
#extract SJLIFE_CCSS variant from these regions
def VAREXT(prsf,sjccssvarf,prematchfile):
    os.system('bgzip -d -c '+sjccssvarf+'|head -n 1 >'+prematchfile+'.tmp')
    os.system('tabix -R '+prsf+'.region '+sjccssvarf+' >>'+prematchfile+'.tmp')
    REFORMFILE(prematchfile)

#split multiallele variants and remove indel
def REFORMFILE(prematchfile):
    fh = open(prematchfile+'.tmp')
    out = open(prematchfile,'w')
    out.write(fh.readline())
    for line in fh:
            l = line.strip().split('\t')
            #remove indel
            if len(l[2]) != 1:
                continue
            for e in l[3].split(','):
                if len(e) != 1:
                    continue
                out.write('\t'.join(l[0:3]+[e])+'\n')
    out.close()
    fh.close()


#hwe extract and filter
#take output of VAREXT as input
def HWEEXT(prsprematchfile,hwefile,preCalhwe):
    os.system('tail -n +2 '+prsprematchfile+'|awk \'BEGIN{FS=OFS="\t"} {print $1,$2,$2}\'  >'+hwefile+'.region')
    os.system('bgzip -d -c '+preCalhwe+'|head -n 1 >'+hwefile)
    os.system('tabix -R '+hwefile+'.region '+preCalhwe+' >>'+hwefile)


#extract variant with hwe p vaule <= than 1e-6 and record in a file
def HWEFILTER(hwepfile,hwepfile_filtered):
    dfhwe = pd.read_table(hwepfile)
    dfhwe = dfhwe[dfhwe['p'] <= 1e-6]
    dfhwe.to_csv(hwepfile_filtered,index=False,header=True,sep='\t')

def DROPVARBYHWE(prematchVar,filteredhwe,var_filtered):
    global hweFailedVar 
    dfprevar = pd.read_table(prematchVar)
    dfprevar_count = dfprevar.shape[0]
    dfhwe = pd.read_table(filteredhwe,usecols=['chr','pos','ref','alt'])
    dfhwe.columns=['chr','pos','ref_allele','alt_allele']
    keys = list(dfhwe.columns.values)
    i1 = dfprevar.set_index(keys).index
    i2 = dfhwe.set_index(keys).index
    dfprevar = dfprevar[~i1.isin(i2)]
    hweDrop = dfprevar_count - dfprevar.shape[0]
    hweFailedVar += hweDrop
    dfprevar.to_csv(var_filtered,index=False,header=True,sep='\t')


def VARMATCH(prematchfile,prsf,matchedfile):
    pgs = pd.read_table(prsf,dtype="string")
    sjlife_ccss = pd.read_table(prematchfile,dtype="string")
    #forward strand
    pgs_sjlife_ccss_merged = pgs.merge(sjlife_ccss, left_on=["chr_name","chr_position","effect_allele","reference_allele"], right_on=["chr","pos","ref_allele","alt_allele"])
    pgs_matched = pgs_sjlife_ccss_merged.loc[:,["chr_name","chr_position","effect_allele","reference_allele","effect_weight"]].to_numpy()
    pgs_sjlife_ccss_merged = pgs.merge(sjlife_ccss, left_on=["chr_name","chr_position","reference_allele","effect_allele"], right_on=["chr","pos","ref_allele","alt_allele"])
    pgs_matched = np.concatenate((pgs_matched, pgs_sjlife_ccss_merged.loc[:,["chr_name","chr_position","effect_allele","reference_allele","effect_weight"]].to_numpy()))
    #reverse strand
    pgs_rev = pgs.copy()
    pgs_effect_alleles = list(map(Seq, pgs.loc[:,"effect_allele"].tolist()))
    pgs_ref_alleles = list(map(Seq, pgs.loc[:,"reference_allele"].tolist()))
    pgs_effect_alleles_rev = list(map(Seq.reverse_complement, pgs_effect_alleles))
    pgs_ref_alleles_rev = list(map(Seq.reverse_complement, pgs_ref_alleles))
    pgs_rev.loc[:,"effect_allele"] = list(map(str, pgs_effect_alleles_rev))
    pgs_rev.loc[:,"reference_allele"] = list(map(str, pgs_ref_alleles_rev))
    pgs_rev = pgs_rev.astype("string")
    pgs_sjlife_ccss_merged = pgs_rev.merge(sjlife_ccss, left_on=["chr_name","chr_position","effect_allele","reference_allele"], right_on=["chr","pos","ref_allele","alt_allele"])
    pgs_matched = np.concatenate((pgs_matched, pgs_sjlife_ccss_merged.loc[:,["chr_name","chr_position","effect_allele","reference_allele","effect_weight"]].to_numpy()))
    pgs_sjlife_ccss_merged = pgs_rev.merge(sjlife_ccss, left_on=["chr_name","chr_position","reference_allele","effect_allele"], right_on=["chr","pos","ref_allele","alt_allele"])
    pgs_matched = np.concatenate((pgs_matched, pgs_sjlife_ccss_merged.loc[:,["chr_name","chr_position","effect_allele","reference_allele","effect_weight"]].to_numpy()))
    np.savetxt(matchedfile, pgs_matched, fmt="%s", delimiter="\t")




### PRS computation
#make a map between sample IDs and sample names

def compute_prs(snpfile):
    prs = {x:{'score':0,'effcount':0} for x in samples}
    # split SNPs by chr and write to temp files 
    chr2tmpsnpfile, pos2snp = parse_snpfile(snpfile)
    print(snpfile)
    tmpfiles = [chr2tmpsnpfile[x] for x in chr2tmpsnpfile]
    # For each SNP chr, run bcftools and filter by maf/callrate
    # pos2snp{'C': ['C', 'T', 0.1261920252377594], 'EAidx': ['1'], 'idx2effallele': {'1': 'C'}, 'effectallelefrequency': 0.1343669250645995, 'gtlst': ['0/0', '0/0', '0/0', '0/1', '0/0']}
    for c in chr2tmpsnpfile:
        tmpchrfile = chr2tmpsnpfile[c]
        run_bcftools(c,tmpchrfile,pos2snp)
    for v in pos2snp:
        pos2snpvar = pos2snp[v]
        if not 'EAidx' in pos2snpvar:
            continue
        radEAidx = pos2snpvar['EAidx'][0]
        radEAnue = pos2snpvar['idx2effallele'][radEAidx]
        radWeight = pos2snpvar[radEAnue][2]
        for gi,gt in enumerate(pos2snpvar['gtlst']):
            if gt == '.':
                prs[samples[gi]]['score'] += (2 * pos2snpvar['effectallelefrequency'] * radWeight)
                continue
            for a in gt.split('/'):
                if a in pos2snpvar['EAidx']:
                    effnue = pos2snpvar['idx2effallele'][a]
                    prs[samples[gi]]['effcount'] += 1
                    prs[samples[gi]]['score'] += pos2snpvar[effnue][2]
    return prs,tmpfiles


def run_bcftools(chr,tmpsnpfile,pos2snp):
    global mafLowVar,lowCallRateVar
    bcfdir = '/research/rgs01/resgen/legacy/gb_customTracks/tp/files/hg38/sjlife/bcf/INFOGT/'
    bcffile = os.path.join(bcfdir,'chr'+chr+'_SJLIFE_CCSS.GT.bcf.gz')
    bcfCall = sp.run("bcftools query -R "+tmpsnpfile+" -f '%CHROM %POS %REF %ALT [%GT ]\\n' -S "+args.sample+".intid "+bcffile,shell=True,stdout=sp.PIPE,stderr=sp.PIPE)
    if not bcfCall.stdout:
        print('bcftools query error: '+bcfCall.stderr.decode('utf-8'))
        sys.exit(1)
    variants = bcfCall.stdout.decode('utf-8').strip().split('\n')
    for v in variants:
        l = v.strip().split(' ')
        snp = pos2snp['.'.join([l[0],l[1]])]
        EAidx,idx2effallele = get_effectalleidx(snp, l[2], l[3])
        missingcallrate, effectallelefrequency,maf = get_effalefreq(l, EAidx)
        #if '.'.join([l[0],l[1]]) == '3.178937643':
        #    print(EAidx,idx2effallele,missingcallrate, effectallelefrequency,maf)
        if args.maf:
            if maf <= 0.01:
                mafLowVar += 1
                continue
        if missingcallrate >= 0.1: #low callrate
                lowCallRateVar += 1
                continue
        snp['EAidx'] = EAidx
        snp['idx2effallele'] = idx2effallele
        snp['effectallelefrequency'] = effectallelefrequency
        snp['gtlst'] = l[4:]



#return missingcallrate, 
#effectallelefrequency,
#maf

def get_effalefreq(l,EAidx):
    EAidx = set(EAidx)
    eacount = 0
    total = 0
    missing = 0
    EAC = {} #count for each allele
    for gt in l[4:]:
        if gt == '.':
            missing += 1
            continue
        for a in gt.split('/'):
            if a in EAidx:
                eacount += 1
            if not a in EAC:
                EAC[a] = 0
            EAC[a] += 1
        total += 2
    #calculate maf
    MAFS = {x:(EAC[x]/total) for x in EAC}
    MAFS_sortkey = sorted(MAFS,key=lambda x: MAFS[x])
    if len(MAFS_sortkey) > 1:
        mafidx = MAFS_sortkey[-2]
        maf = MAFS[mafidx]
        for a in gt.split('/'):
            if a in EAidx:
                eacount += 1
            if not a in EAC:
                EAC[a] = 0
            EAC[a] += 1
        total += 2
    #calculate maf
    MAFS = {x:(EAC[x]/total) for x in EAC}
    MAFS_sortkey = sorted(MAFS,key=lambda x: MAFS[x])
    if len(MAFS_sortkey) > 1:
        mafidx = MAFS_sortkey[-2]
        maf = MAFS[mafidx]
    else:
        maf = 1
    if total == 0:
        print(l[0:4])
        return [0,0,1]
    else:
        return [(missing/len(l[4:])), eacount/total, maf]


def get_effectalleidx(snp,ref,alt):
    alts = alt.split(',')
    effalleleidx = []
    idx2effallele = {}
    if ref in snp:
        effalleleidx.append('0')
        idx2effallele['0'] = ref
    for i,a in enumerate(alts):
        if a in snp:
            effalleleidx.append(str(i+1))
            idx2effallele[str(i+1)] = a
    return effalleleidx,idx2effallele

def parse_snpfile(snpfile):
    chr2snp = {}
    pos2snp = {}
    fh = open(snpfile)
    for line in fh:
            l = line.strip().split('\t')
            chr,pos,effallele,oth,weight = l
            weight = float(weight)
            if not chr in chr2snp:
                chr2snp[chr] = []
            chr2snp[chr].append('\t'.join([chr,pos]))
            cp = '.'.join([chr,pos])
            if not cp in pos2snp:
                pos2snp[cp] = {}
            pos2snp[cp][effallele] = [effallele,oth,weight]
    fh.close()
    chr2tmpsnpfile = {}
    for c in chr2snp:
        out = open(snpfile+'.'+c,'w')
        chr2tmpsnpfile[c] = snpfile+'.'+c
        for ep in chr2snp[c]:
            out.write(ep+'\n')
        out.close()
    return [chr2tmpsnpfile,pos2snp]

###tools
def CKROW(f):
    rowNum = sp.run('wc '+f,shell=True,stdout=sp.PIPE).stdout.decode('utf-8').strip().split(' ')[0]
    if int(rowNum) <= 1:
        return False
    else:
        return True
def OUTPUTSUM(sexChromosomeVar,strandAmbiguousVar,dupVar,lowCallRateVar,hweFailedVar,mafLowVar,outf):
    outk = ['sexChromosomeVar','strandAmbiguousVar','dupVar','lowCallRateVar','hweFailedVar','mafLowVar']
    outv = [sexChromosomeVar,strandAmbiguousVar,dupVar,lowCallRateVar,hweFailedVar,mafLowVar]
    out = open(outf,'w')
    for i,e in enumerate(outk):
        out.write('\t'.join([e,str(outv[i])])+'\n')
    out.close()

import os,re
import gzip
import argparse
from argparse import RawTextHelpFormatter
import pandas as pd
import numpy as np
from Bio.Seq import Seq
import subprocess as sp
import sys

parser = argparse.ArgumentParser(description=script_description,formatter_class=RawTextHelpFormatter)
parser.add_argument('-s','--score',help='PGS score file downloaded from PGS catalog (PGS001238_hmPOS_GRCh38.txt.gz)')
parser.add_argument('-d','--workingDir',help='Working directory, final prs score file will be generated under here')
parser.add_argument('--sample',help='sample file')
parser.add_argument('--maf',help='maf filter. default: off',action="store_true")
parser.add_argument('--hwe',help='hwe file')
parser.add_argument('--ancestry',help='ancestry: asa,ceu or yri')
parser.add_argument('-o','--output',help='output file')
parser.add_argument('--statout',help='summary output')
parser.add_argument('-v','--var',help='SJLIFE_CCSS variants')
parser.add_argument('--matchout',help='output matched variant list. default: off',action="store_true")
args = parser.parse_args()


#maf filter
if args.maf:
    mafMark = '.maf'
else:
    mafMark = ''

##############################
# Initial QC of variant data
##############################
# Discard the following variants: duplicated variants, non-SNP variants (e.g. indels), non-autosomal SNPs, and strand-ambiguous SNPs
PGSID = os.path.split(args.score)[-1].split('_')[0]
qc_PRS_file = os.path.join(args.workingDir,PGSID+'.'+args.ancestry+mafMark)
sexChromosomeVar = 0 #remove variants on sex chromosome
strandAmbiguousVar = 0 #remove ambiguous variants
dupVar = 0 #remove duplicated variants
QCVAR(args.score,qc_PRS_file)
#more QCs:
lowCallRateVar = 0
hweFailedVar = 0
mafLowVar = 0
#check score file row number
rowCK = CKROW(qc_PRS_file)
if not rowCK:
    print('There are no variants: '+qc_PRS_file)
    print('Skip calculating PRS for '+PGSID+'('+args.ancestry+')')
    sys.exit(1)


#########################################
# split SNPs into chunks of 10,000 SNPs
########################################
SNP_CHUNKS = SNPSPLIT(qc_PRS_file)


matchedVariantFile = os.path.join(args.workingDir,'sjlife.ccss.variants.Match')
if args.matchout:
	genMatch = True
else:
	genMatch = False



#############################
# PRS computation
#############################

#sampleID to integer ID
samples = [x.strip() for x in open(args.sample)]
SJLIFE_CCSS_varfile = args.var
PRS = {x:{'score':0,'effcount':0} for x in samples}
for s in SNP_CHUNKS:
    snpchunk = os.path.join(args.workingDir,s)
    newGenFiles = [snpchunk,snpchunk+'.region']
    prematch_var = os.path.join(args.workingDir,'sjlife.ccss.variants.'+args.ancestry+mafMark+'.preMatch.txt') #sjlife.ccss prematch file
    newGenFiles.extend([prematch_var,prematch_var+'.tmp'])
    VAREXT(snpchunk,SJLIFE_CCSS_varfile,prematch_var) #extract variants from SJLIFE_CCSS DATA, generate prematch_var
    if genMatch:
        VARMATCH(prematch_var,snpchunk,matchedVariantFile+'.tmp')
        os.system('cat '+matchedVariantFile+'.tmp >>'+matchedVariantFile)
        os.system('rm -f '+matchedVariantFile+'.tmp')
    #remove variants with hwe p value less than 1e-6
    var_hwe = os.path.join(args.workingDir,'sjlife.ccss.variants.'+args.ancestry+mafMark+'.preMatch.txt.hwe')
    newGenFiles.extend([var_hwe,var_hwe+'.region'])
    HWEEXT(prematch_var,var_hwe,args.hwe) #generate hwe p valur for sjlife_ccss variant
    var_hwe_filtered = var_hwe+'.filtered'
    newGenFiles.append(var_hwe_filtered)
    HWEFILTER(var_hwe,var_hwe_filtered) #generate variant with p value <= 1e-6
    #filter sjlife_ccss variant based on hwe p value (remove var with p value <= 1e-6)
    prematch_var_filtered = prematch_var+'_hwefiltered'
    newGenFiles.append(prematch_var_filtered)
    DROPVARBYHWE(prematch_var,var_hwe_filtered,prematch_var_filtered)
    #variant matching
    matched_vaf_file = os.path.join(args.workingDir,'sjlife.ccss.variants.'+s+mafMark+'.Match') #sjlife.ccss matched file
    #newGenFiles.append(matched_vaf_file)
    VARMATCH(prematch_var_filtered,snpchunk,matched_vaf_file)
    prs_snpchunk,tmpFiles = compute_prs(matched_vaf_file)
    newGenFiles.append(matched_vaf_file)
    newGenFiles.extend(tmpFiles)
    for sam in prs_snpchunk:
        PRS[sam]['score'] += prs_snpchunk[sam]['score']
        PRS[sam]['effcount'] += prs_snpchunk[sam]['effcount']
    for delfile in newGenFiles:
        os.system('rm -f '+delfile)
    print(s+' done!')


#output the total RPS data as a tab-delimited table
out = open(args.output,'w')
out.write('\t'.join(['sampleName','effcount','score'])+'\n')

for s in PRS:
    out.write('\t'.join([s,str(PRS[s]['effcount']),str(PRS[s]['score'])])+'\n')
out.close()
#output summary file
OUTPUTSUM(sexChromosomeVar,strandAmbiguousVar,dupVar,lowCallRateVar,hweFailedVar,mafLowVar,args.statout)
print('done')



Overwriting prs_compute.py


In [3]:
%%writefile gentracktable.py

#!/usr/bin/python3

script_description="""
generate track_table.prs
	input files:
			pgs_all_metadata.xlsx Download from https://www.pgscatalog.org/downloads/
			pgs_traits_data.json Download from https://www.pgscatalog.org/browse/traits/
           
                        
"""
"""
EAS: Asian
EUR: European
"""


import os
import re,sys
import argparse
from argparse import RawTextHelpFormatter
import pandas as pd
import json
import requests
import time
import subprocess as sp

def GETGWASANC(pgs):
	link = 'https://www.pgscatalog.org/rest/score/'+pgs
	f = requests.get(link)
	fjs = json.loads(f.text)
	if not 'ancestry_distribution' in fjs:
		while True:
			time.sleep(10)
			f = requests.get(link)
			fjs = json.loads(f.text)
			if 'ancestry_distribution' in fjs:
				break
	ancestry_distribution = fjs['ancestry_distribution']
	if 'gwas' in ancestry_distribution:
		gwas_ancestry = ancestry_distribution['gwas']['dist']
	elif 'dev' in ancestry_distribution:
		gwas_ancestry = ancestry_distribution['dev']['dist']
	else:
		gwas_ancestry = 'Mixed'
	return gwas_ancestry

def PARSEPOP(gwas_ancestry,pop):
	SELECTPOP = {'Asian','European','African'}
	if gwas_ancestry == 'Mixed':
		return False
	gwas_ancestry = list({pop[x] for x in gwas_ancestry})
	if len(gwas_ancestry) == 1 and gwas_ancestry[0] in SELECTPOP:
		return gwas_ancestry[0]+' ancestry'
	else:
		return False


parser = argparse.ArgumentParser(description=script_description,formatter_class=RawTextHelpFormatter)
parser.add_argument('-m','--meta',help='pgs_all_metadata.xlsx')
parser.add_argument('-t','--traits',help='pgs_traits_data.json')
parser.add_argument('-w','--workingdir',help='working directory where all PGS folders can be found')
parser.add_argument('-o','--output',help='file include PGS scores')
parser.add_argument('-p','--path',help='path to prs scores')
args = parser.parse_args()


POP = {'SAS':'Asian', 'EUR':'European', 'AFR':'African', 'MAO':'Mixed','EAS':'Asian', 'AMR':'Mixed', 'OTH':'Mixed', 'NR':'Mixed', 'ASN':'Asian','MAE':'Mixed','GME':'Mixed'}


#pgs_metadata_table
pgs_metadata_table = pd.read_excel(args.meta,sheet_name="Scores")

#pgs_traits {key:Trait identifier,value:[trait,parent]}
#randomally select one super parent if there are more than one appendicitis in Digestive system disorder&Immune system disorder
pgs_traits_data = json.loads(open(args.traits).read())
th = pgs_traits_data['header'][0] #table_header
pgs_traits = {}
#four traits are overlapped with terms from Kyla
#add "(PRS)" for these 4 traits
dup_traits = {'Cholecystitis','Hypercholesterolemia','Hematuria','Hyperthyroidism'} 
for t in pgs_traits_data['data']:
	tid = t[th[1]].strip().split(':')[0]
	parent = re.findall('[A-Z][^A-Z]*',t[th[2]])
	parent = parent[0]
	trait = t[th[0]]
	if trait in dup_traits:
		trait += ' (PRS)'
	pgs_traits[tid] = [trait,parent]

#pgs2traitParent
pgs2traitParent = {}
for index,row in pgs_metadata_table.iterrows():
	pgs2traitParent[row['Polygenic Score (PGS) ID']] = pgs_traits[row['Mapped Trait(s) (EFO ID)'].split('|')[0]] #randomally select one parent if there are more than one


PGSs = [x for x in os.listdir(args.workingdir) if x.startswith('PGS') and os.path.isdir(os.path.join(args.workingdir,x))]

out = open(args.output,'w')
out.write('\t'.join(['prs','sample_ancestry','prs_name','ancestry','tp_path'])+'\n')
for pgs in PGSs:
	numOfMatVar = sp.run('wc '+os.path.join(args.workingdir,pgs+'/sjlife.ccss.variants.Match'),shell=True,stdout=sp.PIPE).stdout.decode('utf-8').strip().split(' ')[0] 
	if not numOfMatVar or int(numOfMatVar) < 5:
		continue
	track_table_ancestry = ','.join(['Genetic Factors','Polygenic Risk Scores',pgs2traitParent[pgs][1],pgs2traitParent[pgs][0]])
	prs_outancestry = ['All ancestries']
	pgs_gwas_ancestry = GETGWASANC(pgs)
	selpop = PARSEPOP(pgs_gwas_ancestry,POP)
	if selpop:
		prs_outancestry.append(selpop)
		derivFrom = 'Derived from '+selpop
	else:
		derivFrom = 'Derived from Multi-ancestry'
	for a in prs_outancestry:
		prs = ' '.join([pgs,pgs2traitParent[pgs][0],'('+'; '.join([derivFrom,'Computed for '+a])+')'])
		out.write('\t'.join([pgs,a,prs,track_table_ancestry,os.path.join(args.path,pgs)])+'\n')
out.close()


Overwriting gentracktable.py
